# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-NOTEBOOK_CODE_HERE-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Monarchies") 

Consider the following exploratory information need:

> You investigating existing and past historical monarchies

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P1366`   | replaced by    | predicate |
| `wdt:P122`    | basic form of government    | predicate |
| `wd:Q38`      | Italy          | node |
| `wd:Q145`     | United Kingdom           | node |
| `wd:Q7269`    | monarchy           | node |




Also consider

```
?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
```

is the BGP to retrieve all **countries that are some instance of a monarchy**

## Workload Goals


1. Identify the BGP to retrieve states that were monarchies and have been replaced by some other state

2. Identify the BGP for obtaining statistical information of monarchies, e.g., poluation or inception date

3. Which is or was  the country with the highest population?

4. How many countries are or have been monarchies across continents?

5. Analyze the number of countries that were monarchies
 
   5.1 How many sub-types of monarchy exist?  How many instances of exist of each one?
   
   5.2 Are there cities that have been capital of multiple kingdoms?
   
   5.3 Which are the top-3 monarchies with the earliest inception and still existing?


In [1]:
# start your workflow here

In [7]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2344')]


1